In [ ]:
!git clone https://github.com/tomasmuzas/MasterThesis.git

Cloning into 'MasterThesis'...
remote: Enumerating objects: 111, done.
remote: Counting objects: 100% (111/111), done.
remote: Compressing objects: 100% (82/82), done.
remote: Total 111 (delta 45), reused 92 (delta 27), pack-reused 0
Receiving objects: 100% (111/111), 760.25 KiB | 6.61 MiB/s, done.
Resolving deltas: 100% (45/45), done.


In [ ]:
from MasterThesis.read_dataset import *
from MasterThesis.models import *
from MasterThesis.callbacks import *
from tensorflow.keras import optimizers
import numpy as np

In [ ]:
from google.colab import auth
auth.authenticate_user()

# Helper functions

In [ ]:
resolver = tf.distribute.cluster_resolver.TPUClusterResolver(tpu='')
tf.config.experimental_connect_to_cluster(resolver)
tf.tpu.experimental.initialize_tpu_system(resolver)
print("All devices: ", tf.config.list_logical_devices('TPU'))
strategy = tf.distribute.TPUStrategy(resolver)

In [ ]:
def get_dataset():
    training_dataset = read_tf_record_dataset_v2(
        'gs://tomasmuzasmaster2021/dataset/Training',
        tf.keras.layers.Rescaling(scale=1./255),
        image_size = 128,
        batch_size = 1024,
        include_objid = False,
        augment = True,
        drop_remainder=False,
        grayscale=False)
    
    validation_dataset = read_tf_record_dataset_v2(
        'gs://tomasmuzasmaster2021/dataset/Validation',
        tf.keras.layers.Rescaling(scale=1./255),
        image_size = 128,
        batch_size = 16,
        include_objid = False,
        augment=False,
        drop_remainder=True,
        grayscale=False)
    return (training_dataset.repeat(), 247, validation_dataset.repeat(), 1984)

In [ ]:
import os

def perform_training(model_factory, training_parameters, starting_training=0):
  tf.keras.backend.clear_session()

  model_name = f"{training_parameters['model_name']}"
  training_dataset, training_steps, validation_dataset, validation_steps = get_dataset()

  path = f"./drive/MyDrive/MTD/Models/{model_name}"
  os.makedirs(path, exist_ok=True)
  f = open(path + "/training_parameters.txt", "w")
  f.write(str(training_parameters))
  f.close()

  for training in range(starting_training, training_parameters["trainings"]):
      print(f"--------------------------------------- TRAINING {training + 1} {training_parameters['model_name']} ---------------------------------------")
      print("starting with new model.")

      with strategy.scope():
        model = model_factory(training_parameters["classes"], training_parameters["image_size"])
        sgd = optimizers.Adam(learning_rate= training_parameters["learninig_rate"], beta_1=0.9, beta_2=0.999, epsilon=1e-08)
        model.compile(
            loss=tf.keras.losses.BinaryCrossentropy(),
            optimizer=sgd,
            steps_per_execution = 1,
            metrics=[tensorflow.keras.metrics.BinaryAccuracy()])

        callbacks = [
          BestAccuracyCallback(training_parameters["monitor"], model_name, f"training_{training + 1}"),
          # LoggingCallback(model_name, f"training_{training + 1}"),
          tf.keras.callbacks.EarlyStopping(monitor=training_parameters["monitor"], patience=20, mode='max')
        ]

        history = model.fit(
            x= training_dataset,
            validation_data = validation_dataset,
            epochs = training_parameters["epochs"],
            verbose = 1,
            steps_per_epoch = training_steps,
            validation_steps = validation_steps,
            callbacks= callbacks,
            shuffle=True,
            class_weight=training_parameters["weights"],
            workers=32)

        f = open(path + f"/training_{training + 1}/best.txt", "w")
        f.write(str(max(history.history[training_parameters["monitor"]])))
        f.close()
        


In [ ]:
print(tf.__version__)

@tf.function
def random_invert_horizontally(x, y, p=0.5):
  print(x.shape)
  print(y)
  print(tf.math.equal(y, 1) == True)
  if  tf.random.uniform([]) < p and tf.math.equal(y, 1) == True:
    x = tf.image.flip_left_right(x)
  else:
    x
  return x

@tf.function
def random_invert_vertically(x, y, p=0.5):
  if  tf.random.uniform([]) < p and tf.math.equal(y, 1) == True:
    x = tf.image.flip_up_down(x)
  else:
    x
  return x

@tf.function
def random_rotate(x, y, p=0.5):
  if  tf.random.uniform([]) < p and tf.math.equal(y, 1) == True:
    x = tf.image.rot90(x, k = tf.random.uniform([], minval=1, maxval=4, dtype=tf.int32))
  else:
    x
  return x


@tf.function
def random_zoom(x, y, p=0.5):
  if  tf.random.uniform([]) < p and tf.math.equal(y, 1) == True:
    x = tf.image.crop_to_bounding_box(x, 10, 10, 108, 108)
    x = tf.image.resize(x, (128,128))
  else:
    x
  return x


def read_tf_record_dataset_v2(path, preprocessing_function, image_size, batch_size, augment = False, include_objid = False, drop_remainder = True, grayscale = False):
  filenames = tf.io.gfile.glob(path + "/*.tfrec")
  dataset4 = tf.data.TFRecordDataset(filenames, num_parallel_reads=AUTO)
  dataset4 = dataset4.map(lambda x: read_tfrecord(x, image_size), num_parallel_calls=AUTO)
  if (include_objid):
    dataset4 = dataset4.map(lambda image, class_num, label, objid, one_hot_class: (image, class_num, objid))
    dataset4 = dataset4.map(lambda x, y, z: (tf.cast(x, tf.float32), y, z), num_parallel_calls=AUTO)
    dataset4 = dataset4.map(lambda x, y, z: (preprocessing_function(x), y, z), num_parallel_calls=AUTO)
    if(augment):
      dataset4 = dataset4.map(lambda x,y,z : (random_zoom(x, y), y, z), num_parallel_calls=AUTO)
      dataset4 = dataset4.map(lambda x,y,z : (random_invert_horizontally(x, y), y, z), num_parallel_calls=AUTO)
      dataset4 = dataset4.map(lambda x,y,z : (random_invert_vertically(x, y), y, z), num_parallel_calls=AUTO)
      dataset4 = dataset4.map(lambda x,y,z : (random_rotate(x, y), y, z), num_parallel_calls=AUTO)
      
  else:
    dataset4 = dataset4.map(lambda image, class_num, label, objid, one_hot_class: (image, class_num))
    dataset4 = dataset4.map(lambda x, y: (tf.cast(x, tf.float32), y), num_parallel_calls=AUTO)
    dataset4 = dataset4.map(lambda x, y: (preprocessing_function(x), y), num_parallel_calls=AUTO)
    if(augment):
      dataset4 = dataset4.map(lambda x,y : (random_zoom(x, y), y), num_parallel_calls=AUTO)
      dataset4 = dataset4.map(lambda x,y : (random_invert_horizontally(x, y), y), num_parallel_calls=AUTO)
      dataset4 = dataset4.map(lambda x,y : (random_invert_vertically(x, y), y), num_parallel_calls=AUTO)
      dataset4 = dataset4.map(lambda x,y : (random_rotate(x, y), y), num_parallel_calls=AUTO)
      

  return dataset4.batch(batch_size, drop_remainder=drop_remainder).prefetch(AUTO)

2.7.0


# All models

In [ ]:
from tensorflow.keras.layers import Dropout, Flatten, BatchNormalization

NUMBER_OF_CHANNELS = 3

def Dieleman(classes, image_size):
  model = Sequential(name="Dieleman")
  model.add(Input(shape=(image_size, image_size, NUMBER_OF_CHANNELS)))
  model.add(Conv2D(filters=32, kernel_size=6, activation='relu'))
  model.add(BatchNormalization())   
  model.add(MaxPool2D(2))
  model.add(Conv2D(filters=64, kernel_size=5, activation='relu'))
  model.add(BatchNormalization())  
  model.add(MaxPool2D(2))
  model.add(Conv2D(filters=128, kernel_size=3, activation='relu'))
  model.add(Conv2D(filters=128, kernel_size=3, activation='relu'))
  model.add(BatchNormalization())
  model.add(MaxPool2D(2))
  model.add(Flatten())
  model.add(Dropout(0.5))
  model.add(Dense(256,activation='relu'))
  model.add(Dense(256,activation='relu'))
  model.add(Dense(classes, activation='sigmoid'))
  return model


def Cavanagh(classes, image_size):
  model = Sequential(name="Cavanagh")
  model.add(Input(shape=(image_size, image_size, NUMBER_OF_CHANNELS)))
  model.add(Conv2D(filters=32, kernel_size=7, activation='relu'))
  model.add(BatchNormalization())   
  model.add(MaxPool2D(2))
  model.add(Conv2D(filters=64, kernel_size=5, activation='relu'))
  model.add(Conv2D(filters=64, kernel_size=5, activation='relu'))
  model.add(BatchNormalization())  
  model.add(MaxPool2D(2))
  model.add(Conv2D(filters=128, kernel_size=3, activation='relu'))
  model.add(BatchNormalization())
  model.add(MaxPool2D(2))
  model.add(Flatten())
  model.add(Dropout(0.5))
  model.add(Dense(256,activation='relu'))
  model.add(Dense(256,activation='relu'))
  model.add(Dense(classes, activation='sigmoid'))
  return model

models = [
    {'name': 'Cavanagh', 'func': Cavanagh, 'starting_training': 0},
    {'name': 'Dieleman', 'func': Dieleman, 'starting_training': 0},
    {'name': 'ResNet50', 'func': create_ResNet50_model, 'starting_training': 0},
    {'name': 'InceptionV3', 'func': create_InceptionV3_model, 'starting_training': 0},
    {'name': 'InceptionResNetV2', 'func': create_InceptionResNetV2_model, 'starting_training': 0},
]

for model in models:
  training_parameters = {
        "model_name": "Models/" + model["name"],
        "image_size": 128,
        "learninig_rate": 1e-4,
        "classes": 1,
        "weights": None,
        "epochs": 500,
        "trainings": 3,
        "monitor": 'val_binary_accuracy'
    }

  perform_training(
      model["func"],
      training_parameters,
      starting_training=model["starting_training"])

# Hyperparameter search

In [ ]:
!pip install -q -U keras-tuner

     |████████████████████████████████| 98 kB 3.4 MB/s 


In [ ]:
import keras_tuner as kt
from keras_tuner.applications import HyperResNet
from tensorflow.keras.layers import Dropout, Flatten, BatchNormalization

def TestModel(hp):
  model = Sequential(name="Dieleman")
  model.add(Input(shape=(128, 128, 3)))
  model.add(Conv2D(filters=hp.Int("conv_1_filters", 16, 32, 16), kernel_size=6, activation='relu'))
  model.add(BatchNormalization())   
  model.add(MaxPool2D(2))
  model.add(Conv2D(filters=hp.Int("conv_2_filters", 16, 64, 16), kernel_size=5, activation='relu'))
  model.add(BatchNormalization())  
  model.add(MaxPool2D(2))
  conv3_filters = hp.Int("conv_3_filters", 32, 128, 16)
  model.add(Conv2D(filters=conv3_filters, kernel_size=3, activation='relu'))
  model.add(Conv2D(filters=conv3_filters, kernel_size=3, activation='relu'))
  model.add(BatchNormalization())
  model.add(MaxPool2D(2))
  model.add(Flatten())
  # model.add(Dropout(hp.Float("dropout", min_value=0.2, max_value=0.8, step=0.1)))
  model.add(Dense(hp.Int("dense_1_units", 16, 64, 16),activation='relu'))
  model.add(Dense(hp.Int("dense_2_units", 16, 64, 16),activation='relu'))
  model.add(Dense(1, activation='sigmoid'))

  # learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])
  learning_rate = 1e-4
  model.compile(
      optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate),
      loss=tf.keras.losses.BinaryCrossentropy(),
      metrics=[tensorflow.keras.metrics.BinaryAccuracy()]
  )
  return model


  # learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])
  learning_rate = 1e-4
  model.compile(
      optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate),
      loss=tf.keras.losses.BinaryCrossentropy(),
      metrics=[tensorflow.keras.metrics.BinaryAccuracy()]
  )
  return model

tuner = kt.Hyperband(
    TestModel,
    objective="val_binary_accuracy",
    max_epochs=35,
    factor=2,
    # max_trials=8,
    # executions_per_trial=2,
    overwrite=True,
    directory="gs://tomasmuzasmaster2021/hyperparams",
    project_name="DielemanDropoutTest",
    distribution_strategy=strategy
)

training_dataset, training_steps, validation_dataset, validation_steps = get_dataset()

callbacks = [
  # BestAccuracyCallback(training_parameters["monitor"], model_name, f"training_{training + 1}"),
  tf.keras.callbacks.EarlyStopping(monitor='val_binary_accuracy', patience=15, mode='max')
]

tuner.search(
    x= training_dataset,
    validation_data = validation_dataset,
    epochs=50,
    callbacks=callbacks,
    steps_per_epoch = training_steps,
    validation_steps = validation_steps,
    shuffle=True
)

Trial 115 Complete [00h 05m 22s]
val_binary_accuracy: 0.9360193610191345

Best val_binary_accuracy So Far: 0.9498172402381897
Total elapsed time: 06h 35m 22s

Search: Running Trial #116

Hyperparameter    |Value             |Best Value So Far 
conv_1_filters    |16                |16                
conv_2_filters    |16                |16                
conv_3_filters    |112               |80                
dense_1_units     |32                |48                
dense_2_units     |48                |48                
tuner/epochs      |9                 |35                
tuner/initial_e...|5                 |18                
tuner/bracket     |4                 |5                 
tuner/round       |2                 |5                 
tuner/trial_id    |e156fdae9617435...|e10aacffa8520e7...

Epoch 6/9
  6/247 [..............................] - ETA: 19s - loss: 0.4944 - binary_accuracy: 0.7660WARNING:tensorflow:Callback method `on_train_batch_end` is slow compared to the bat

247/247 [==============================] - 81s 286ms/step - loss: 0.2255 - binary_accuracy: 0.9081 - val_loss: 0.5684 - val_binary_accuracy: 0.7584
Epoch 7/9
247/247 [==============================] - 88s 356ms/step - loss: 0.1679 - binary_accuracy: 0.9320 - val_loss: 0.3129 - val_binary_accuracy: 0.8850
Epoch 8/9
247/247 [==============================] - 94s 382ms/step - loss: 0.1541 - binary_accuracy: 0.9380 - val_loss: 0.1588 - val_binary_accuracy: 0.9354
Epoch 9/9
 75/247 [========>.....................] - ETA: 44s - loss: 0.1475 - binary_accuracy: 0.9403